<a href="https://colab.research.google.com/github/krusehsbi/LESS/blob/main/less.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LESS: Litter Examination Using Sparse Samples

# Data

TACO-Dataset: [tacodataset.org](http://tacodataset.org)